# class drake::geometry::QueryObject< T >

In [1]:
# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, SnoptSolver, IpoptSolver,  
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
    )

In [2]:
h = 1e-3
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=h)
parser = Parser(plant)

In [3]:
dir1 = "/Users/xiao/0_codes/ICBM_drake/models/objects"
dir2 = "/Users/xiao/0_codes/ICBM_drake/models/ycb/sdf/"
table_file = os.path.join(dir1, "table_top.sdf")
panda_file = "/Users/xiao/0_codes/ICBM_drake/models/franka_description/urdf/panda_arm_hand.urdf"
box_file = os.path.join(dir2, "003_cracker_box.sdf")
table_modelInstance = parser.AddModels(table_file)[0]  # this return pydrake.multibody.tree.ModelInstanceIndex
panda_modelInstance = parser.AddModels(panda_file)[0]
box_modelInstance = parser.AddModels(box_file)[0]

table_top_frame = plant.GetFrameByName("table_top_center")
robot_base = plant.GetFrameByName("panda_link0")

X_W_table = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],]),
  p=[0.0, 0.0, 0.0]
)

plant.WeldFrames(plant.world_frame(), table_top_frame, X_W_table)
plant.WeldFrames(table_top_frame, robot_base, RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.]))
plant.Finalize()
plant.set_name("table_with_box")
diagram = builder.Build()
context = diagram.CreateDefaultContext()
mutable_context = plant.GetMyContextFromRoot(context)

<div class="alert alert-block alert-success">
<b>Querying the scene geometry based on context at $t_k$:</b> 1. create mutable_contexts from root_context for each time step; 2. at $t_k$, using its corresponding mutable_context to create a query_port; 3. create query_object, which has some useful member functions; 4. create the inspector = query_object.inspector() to extract informations from the scene geometries.
</div>

In [5]:
query_port = plant.get_geometry_query_input_port()

In [9]:
if query_port.HasValue(mutable_context):
    query_object = query_port.Eval(mutable_context)

In [10]:
signed_distance_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints()

In [26]:
print(len(signed_distance_pairs))

2165


In [16]:
print(signed_distance_pairs[0].id_A)
print(signed_distance_pairs[0].id_B)

<GeometryId value=21>
<GeometryId value=282>


In [23]:
# using the inspector to get info on the pairs
inspector = query_object.inspector()
frameA_id = inspector.GetFrameId(signed_distance_pairs[0].id_A)
frameB_id = inspector.GetFrameId(signed_distance_pairs[1].id_B)

bodyA = plant.GetBodyFromFrameId(frameA_id)
bodyB = plant.GetBodyFromFrameId(frameB_id)

In [24]:
type(bodyA)

pydrake.multibody.tree.RigidBody

In [27]:
print(bodyA)
print(bodyB)

<RigidBody name='table_top_link' index=1 model_instance=2>
<RigidBody name='base_link_cracker' index=14 model_instance=4>


__A variant of ComputeSignedDistancePairwiseClosestPoints() which computes the signed distance (and witnesses) between a specific pair of geometries indicated by id.__ 

In [28]:
"""
# This could be used for getting pairs between finger and box
query_object.ComputeSignedDistancePairClosestPoints(GeometryId geometry_id_A,
                                                    GeometryId geometry_id_B)
"""                                                     """
# maybe use this 
query_object.ComputeSignedDistanceToPoint()
"""

# get link 7 in {W}
link7_frame = plant.GetFrameByName("panda_link7")

In [30]:
print(link7_frame)

<BodyFrame name='panda_link7' index=11 model_instance=3>


In [35]:
link7_frame.CalcPoseInBodyFrame(mutable_context)

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)

In [37]:
link7_frame.CalcPoseInWorld(mutable_context)

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, -1.0, -9.793177720293495e-12],
    [0.0, 9.793177720293495e-12, -1.0],
  ]),
  p=[0.088, -0.39999999999966707, 1.033],
)

<div class="alert alert-block alert-success">
<b>I know what to do next!!:</b> Manually choose a point on the finger in {link8}. Then manually choose three points of the box to give surface normal. Something useful relevant to contact constraints: https://drake.mit.edu/doxygen_cxx/group__contact__surface__constraints.html
</div>

__this will be useful!__ #include "drake/geometry/proximity/calc_distance_to_surface_mesh.h" and "proximity_utilities.h" These are what I will be doing manually.

In [41]:
box_frame = plant.GetFrameByName("base_link_cracker")
print(f"box fram in W: {box_frame.CalcPoseInWorld(mutable_context)}")

box fram in W: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)


In [43]:
panda_modelInstance

ModelInstanceIndex(3)

In [44]:
plant.world_frame()

<BodyFrame name='world' index=0 model_instance=0>

In [47]:
plant.world_frame().CalcPoseInWorld(mutable_context)

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)

In [431]:
""" a function to extract collision points from the box
    These points are expressed in {box}
"""
def getPointsFromCollisionBox(modelfile):
    points = []
    with open(modelfile, 'r') as f:
        model_file = f.read()
    model_data = bs(model_file, "xml")
    box_collision_points = model_data.find_all('collision') # this gathers all <collision> tags in the model.
    # print(box_collision_points)
    for ele in box_collision_points:
        if ele["name"].split('_')[0] == "point":
            coords = ele.text.strip().split(' ')
            points.append([coords[0], coords[1], coords[2]])
    return np.array(points).astype(float)
    
vertices = getPointsFromCollisionBox(box_file)
vertices

array([[ 0.0795,  0.1042,  0.0334],
       [ 0.0795, -0.1042,  0.0334],
       [-0.0795,  0.1042,  0.0334],
       [-0.0795, -0.1042,  0.0334],
       [ 0.0795,  0.1042, -0.0334],
       [ 0.0795, -0.1042, -0.0334],
       [-0.0795,  0.1042, -0.0334],
       [-0.0795, -0.1042, -0.0334]])

__Now I can compute the surfaces normal, then project a point in hand onto the surface.__
__Next up: get a point in left finger, then express in {W}__
__Then, compute the relative velocity at contact point {C}__

In [331]:
# get the transformation from {rightfinger} to {link7}
rightfinger_frame = plant.GetFrameByName("panda_rightfinger")
link7_frame = plant.GetFrameByName("panda_link7")
X_w_rightfinger = rightfinger_frame.CalcPoseInWorld(mutable_context)
X_w_link7 = link7_frame.CalcPoseInWorld(mutable_context)
X_link7_p = X_w_link7.inverse().multiply(X_w_rightfinger)

# from the urdf collision bodies get a roughly exstimated points, express in {link7}
p_rf = np.array([[0], [-0.007], [0.045]])
p_link7_rf = X_link7_p.multiply(p_rf)
print(f"A point of the right finger expressed in panda_link7 is:\n {p_link7_rf}")

A point of the right finger expressed in panda_link7 is:
 [[-0.00494975]
 [-0.00494975]
 [ 0.2104    ]]


In [361]:
frameIds = inspector.GetAllFrameIds()
# plant.GetBodyFromFrameId(frameIds)
def getFrameIdByLinkName(linkName, frameIds):
    for id in frameIds:
        if plant.GetBodyFromFrameId(id).name() == linkName:
            return id

<div class="alert alert-block alert-danger">
<b>Just realized!!:</b> I should not rigidly follow the C++ sliding_friction_complementarity_constraint.cc line 211-216. Because in Drake's pipeline, the contact points are detected by SceneGraph(), which are registered as points in the colliding geometries. Then it makes sense to transform the points into body frame. I can just do this manually as I know the contacts in bodies and their poses. 
</div>

# Continue here!

In [414]:
# micmicing line 211-221 in sliding_fricition_complementarity_constraint.cc
pA_in_geom_finger = np.array([[0], [-0.007], [0.045]])
rightfinger_frameId = getFrameIdByLinkName("panda_rightfinger", frameIds)
rightfinger_pose = rightfinger_frame.CalcPoseInBodyFrame(mutable_context)
pA_in_rightfinger_frame = rightfinger_pose @ pA_in_geom_finger
pA_in_w_1 = plant.CalcRelativeTransform(mutable_context, plant.world_frame(), rightfinger_frame).rotation() @ pA_in_rightfinger_frame
pA_in_w_1

array([[-0.00494975],
       [ 0.00494975],
       [-0.045     ]])

In [435]:
# a function that returns the surface normals of a geometry expressed in {W}
def getNormalsOfBoxInGeom(vertices):
    """
    Top surface is indexed as (clockwise): 0, 1, 3, 2
    bottom surface (clockwise): 4, 5, 7, 6
    """
    # edges from vertices from far corners, common at index 0. 
    # the edges are: 01, 04, 02.
    edge01 = vertices[1] - vertices[0]
    edge04 = vertices[4] - vertices[0]
    edge02 = vertices[2] - vertices[0]
    
    # edges from vertices, common at index 7. 
    # the edges are: 75, 76, 73
    edge75 = vertices[5] - vertices[7]
    edge76 = vertices[6] - vertices[7]
    edge73 = vertices[3] - vertices[7]
    
    # get normals using edges (right hand rule, cross product). surface normals always go outwards
    normals = np.zeros((6, 3))
    normals[0, :] = np.cross(edge04, edge01)/np.linalg.norm(np.cross(edge04, edge01)) # surface 04x01
    normals[1, :] = np.cross(edge04, edge02)/np.linalg.norm(np.cross(edge04, edge02)) # surface 04x02
    normals[2, :] = np.cross(edge02, edge01)/np.linalg.norm(np.cross(edge02, edge01)) # surface 02x01
    normals[3, :] = np.cross(edge75, edge73)/np.linalg.norm(np.cross(edge75, edge73)) # surface 75x73
    normals[4, :] = np.cross(edge76, edge75)/np.linalg.norm(np.cross(edge76, edge75)) # surface 76x75
    normals[5, :] = np.cross(edge73, edge76)/np.linalg.norm(np.cross(edge73, edge76)) # surface 73x76

    return normals

__Pay special attention here that only the rotation part is used. Because we need to know the contact point Cb (or the witness point) orientation in the current link frame expressed in {W}. Drake's pipline detects contact points using .inspector() in SceneGraph() which produce the contacts in pairs of contact bodies by pydrake.geometry.SignedDistancePair() class. Each body in the pair is referenced using GeometryId and the witness point coordinats is respecting the Geometry frame (NOT the current body frame!!). Then current body frame is required to transform the point to body frame using only the rotation is needed because the both frames are at the same location but different orientations. In other words, we want it to rotate in place.__

In [419]:
# same as in last block
pCb_in_rightfinger = np.array([[0], [-0.007], [0.045]])
rightfinger_pose = rightfinger_frame.CalcPoseInWorld(mutable_context)
p_rfCb_in_w = rightfinger_pose.rotation() @ pCb_in_rightfinger

In [422]:
# get spatial velocity of right finger measured in box frame, expressed in {W}
V_box_rf_w = rightfinger_frame.CalcSpatialVelocity(mutable_context, box_frame, plant.world_frame())

# v_box_Cb_W is the sliding velocity of witness point Cb measured from the box, expressed in the world frame.
v_box_Cb_w = V_box_rf_w.Shift(p_rfCb_in_w).translational();
v_box_Cb_w

array([0., 0., 0.])

__choosing an arbitrary surface for the time being. Later maybe choosing the closest one.__

In [440]:
"""
Now, project the sliding velocity v_box_Cb_W to the tangential plane. 
Then need to know the surface normal, nhat_box_Cb_w. """
#nhat_BA_W = signed_distance_pair.nhat_BA_W;
#const Vector3<AutoDiffXd> v_sliding_ACb_W = (Eigen::Matrix3d::Identity() - nhat_BA_W * nhat_BA_W.transpose()) * v_ACb_W;
nhats = getNormalsOfBoxInGeom(vertices) # nhats in {body} frame

# Now we are arbitrarily choosing a surfce to work with for the time being.
nhat_box_to_finger = nhats[0]

# express the normal in {b} in {s}
nhat_box_to_finger_w =  box_frame.CalcPoseInWorld(mutable_context).rotation() @ nhat_box_to_finger

# projecting the sliding velocity onto the contact tangential plane (here one of the surface of the box)
v_sliding_box_Cb_w = v_box_Cb_w - np.dot(v_box_Cb_w, nhat_box_to_finger_w) * nhat_box_to_finger_w
v_sliding_box_Cb_w

array([0., 0., 0.])